In [2]:
import tensortrade.env.default as default
from tensortrade.oms.exchanges import Exchange
from tensortrade.feed import Stream
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.oms.instruments import Instrument
from tensortrade.agents import DQNAgent
from tensortrade.env.default.actions import BSH, ManagedRiskOrders
from tensortrade.env.default.rewards import RiskAdjustedReturns
from tensortrade.env.default.renderers import PlotlyTradingChart
from all_indicators import get_all_stock_indicators
from sklearn.preprocessing import MinMaxScaler
import multiprocessing
import pandas as pd
import numpy as np

pd.options.mode.use_inf_as_na = True

B:\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [46]:
TKO_data = get_all_stock_indicators('TKO.PA')
STM_data = get_all_stock_indicators('STM.PA')
THEP_data = get_all_stock_indicators('THEP.PA')
BIM_data = get_all_stock_indicators('BIM.PA')
ERF_data = get_all_stock_indicators('ERF.PA')
TRI_data = get_all_stock_indicators('TRI.PA')
VIRP_data = get_all_stock_indicators('VIRP.PA')
AI_data = get_all_stock_indicators('AI.PA')
SU_data = get_all_stock_indicators('SU.PA')
LTA_data = get_all_stock_indicators('LTA.PA')

dropped Index(['trend_psar_up', 'trend_psar_down', 'alpha26', 'alpha45', 'Sortino 10'], dtype='object')
dropped Index(['trend_psar_up', 'trend_psar_down', 'alpha26'], dtype='object')
dropped Index(['trend_psar_up', 'trend_psar_down', 'alpha26', 'alpha45'], dtype='object')
dropped Index(['trend_psar_up', 'trend_psar_down', 'alpha26'], dtype='object')
dropped Index(['trend_psar_up', 'trend_psar_down', 'alpha26'], dtype='object')
dropped Index(['trend_psar_up', 'trend_psar_down', 'alpha26'], dtype='object')
dropped Index(['trend_psar_up', 'trend_psar_down', 'alpha26', 'alpha45'], dtype='object')
dropped Index(['trend_psar_up', 'trend_psar_down', 'alpha26', 'alpha94'], dtype='object')
dropped Index(['trend_psar_up', 'trend_psar_down', 'alpha26', 'alpha94'], dtype='object')
dropped Index(['trend_psar_up', 'trend_psar_down', 'alpha15', 'alpha26', 'alpha45'], dtype='object')


In [55]:
selected_var = pd.read_excel('feature_selection.xlsx', sheet_name='ALL')

def drop_corr_col(stock_data, ticker):
    relevant_cols = selected_var[ticker].dropna()
    relevant_cols = relevant_cols[relevant_cols!='Sortino 10']
    filtered_data = stock_data.copy().drop(columns=['Open', 'High', 'Low', 'Close', 'Volume','Returns','Log Returns'])
    filtered_data = filtered_data[relevant_cols]
    corr_matrix = filtered_data.corr()
    upperMatrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    corrFeatures = [column for column in upperMatrix.columns if any(upperMatrix[column] > 0.95)]
    print(corrFeatures)
    return pd.concat([filtered_data.drop(columns=corrFeatures),stock_data[['Open', 'High', 'Low', 'Close', 'Volume','Returns','Log Returns']]],axis=1)

In [56]:
selected_STM_data = drop_corr_col(STM_data, 'STM')
selected_THEP_data = drop_corr_col(THEP_data, 'THEP')
selected_BIM_data = drop_corr_col(BIM_data, 'BIM')
selected_TKO_data = drop_corr_col(TKO_data, 'TKO')
selected_ERF_data = drop_corr_col(ERF_data, 'ERF')
selected_TRI_data = drop_corr_col(TRI_data, 'TRI')
selected_VIRP_data = drop_corr_col(VIRP_data, 'VIRP')
selected_AI_data = drop_corr_col(AI_data, 'AI')
selected_SU_data = drop_corr_col(SU_data, 'SU')
selected_LTA_data = drop_corr_col(LTA_data, 'LTA')

['trend_vortex_ind_diff', 'Sharpe 14', 'Sharpe 18', 'Sharpe 16', 'Beta 24']
['Sharpe 12']
['Sharpe 18']
['Sharpe 14', 'Sharpe 16', 'Sharpe 18', 'Sortino 24', 'Sortino 26', 'Beta 24']
['Sharpe 126', 'Sortino 24', 'Sharpe 16']
[]
['Sortino 22']
['Sharpe 16', 'Sortino 18']
['Sortino 16', 'Sharpe 18']
['volatility_bbh', 'Sortino 126', 'trend_ichimoku_b', 'rsi10']


In [58]:
from sklearn.model_selection import train_test_split

def split_data(data):
    X = data.copy().drop(columns=['Log Returns','Returns'])
    y = data.copy()['Returns']

    X_train_test, X_valid, y_train_test, y_valid = \
        train_test_split(X, y, train_size=0.8, test_size=0.2, shuffle=False)

    X_train, X_test, y_train, y_test = \
        train_test_split(X_train_test, y_train_test, train_size=0.7, test_size=0.3, shuffle=False)

    return X_train, X_test, X_valid, y_train, y_test, y_valid

TKO_X_train, TKO_X_test, TKO_X_valid, TKO_y_train, TKO_y_test, TKO_y_valid = split_data(selected_TKO_data)
STM_X_train, STM_X_test, STM_X_valid, STM_y_train, STM_y_test, STM_y_valid = split_data(selected_STM_data)
THEP_X_train, THEP_X_test, THEP_X_valid, THEP_y_train, THEP_y_test, THEP_y_valid = split_data(selected_THEP_data)
BIM_X_train, BIM_X_test, BIM_X_valid, BIM_y_train, BIM_y_test, BIM_y_valid = split_data(selected_BIM_data)
ERF_X_train, ERF_X_test, ERF_X_valid, ERF_y_train, ERF_y_test, ERF_y_valid = split_data(selected_ERF_data)
TRI_X_train, TRI_X_test, TRI_X_valid, TRI_y_train, TRI_y_test, TRI_y_valid = split_data(selected_TRI_data)
VIRP_X_train, VIRP_X_test, VIRP_X_valid, VIRP_y_train, VIRP_y_test, VIRP_y_valid = split_data(selected_VIRP_data)
AI_X_train, AI_X_test, AI_X_valid, AI_y_train, AI_y_test, AI_y_valid = split_data(selected_AI_data)
SU_X_train, SU_X_test, SU_X_valid, SU_y_train, SU_y_test, SU_y_valid = split_data(selected_SU_data)
LTA_X_train, LTA_X_test, LTA_X_valid, LTA_y_train, LTA_y_test, LTA_y_valid = split_data(selected_LTA_data)

In [59]:
TKO_X_train.to_pickle('TKO_X_train.pickle')
STM_X_train.to_pickle('STM_X_train.pickle')
THEP_X_train.to_pickle('THEP_X_train.pickle')
BIM_X_train.to_pickle('BIM_X_train.pickle')
ERF_X_train.to_pickle('ERF_X_train.pickle')
TRI_X_train.to_pickle('TRI_X_train.pickle')
VIRP_X_train.to_pickle('VIRP_X_train.pickle')
AI_X_train.to_pickle('AI_X_train.pickle')
SU_X_train.to_pickle('SU_X_train.pickle')
LTA_X_train.to_pickle('LTA_X_train.pickle')

In [60]:
def separate_render_features(stock_data, ticker):
    #ohlc prices for render
    stock_ohlc = stock_data[['Open','High','Low','Close','Volume']].copy()
    stock_ohlc['date'] = stock_ohlc.index + pd.DateOffset(hours=2)
    stock_ohlc = stock_ohlc.add_prefix(f"{ticker}:")
    
    #all features to train from + minmax scaling
    scaler = MinMaxScaler()
    stock_features = stock_data.copy()
    stock_features = stock_features.add_prefix(f"{ticker}:")
    scaler.fit(stock_features)
    stock_features_scaled = pd.DataFrame(scaler.fit_transform(stock_features), columns = stock_features.columns, index = stock_features.index)
    return stock_ohlc, stock_features_scaled

def get_price_stream(stock_renders, tickers): #list
    stock_price_stream_list = []
    for i in range(len(stock_renders)):
        stock_price_stream_list.append(Stream.source(list(stock_renders[i][f"{tickers[i]}:Close"]), dtype="float").rename(f"EUR-{tickers[i].replace('.PA', '')}"))
    return stock_price_stream_list

In [61]:
TKO_data = pd.read_pickle('TKO_X_train.pickle')
STM_data = pd.read_pickle('STM_X_train.pickle')
THEP_data = pd.read_pickle('THEP_X_train.pickle')
BIM_data = pd.read_pickle('BIM_X_train.pickle')
ERF_data = pd.read_pickle('ERF_X_train.pickle')
TRI_data = pd.read_pickle('TRI_X_train.pickle')
VIRP_data = pd.read_pickle('VIRP_X_train.pickle')
AI_data = pd.read_pickle('AI_X_train.pickle')
SU_data = pd.read_pickle('SU_X_train.pickle')
LTA_data = pd.read_pickle('LTA_X_train.pickle')

In [62]:
TKO_render, TKO_scaled_features = separate_render_features(TKO_data, 'TKO.PA')
STM_render, STM_scaled_features = separate_render_features(STM_data, 'STM.PA')
THEP_render, THEP_scaled_features = separate_render_features(THEP_data, 'THEP.PA')
BIM_render, BIM_scaled_features = separate_render_features(BIM_data, 'BIM.PA')   
ERF_render, ERF_scaled_features = separate_render_features(ERF_data, 'ERF.PA')
TRI_render, TRI_scaled_features = separate_render_features(TRI_data, 'TRI.PA')
VIRP_render, VIRP_scaled_features = separate_render_features(VIRP_data, 'VIRP.PA')
AI_render, AI_scaled_features = separate_render_features(AI_data, 'AI.PA')       
SU_render, SU_scaled_features = separate_render_features(SU_data, 'SU.PA')
LTA_render, LTA_scaled_features = separate_render_features(LTA_data, 'LTA.PA')

In [63]:
price_streams = get_price_stream([TKO_render, STM_render, THEP_render, BIM_render, ERF_render, TRI_render, VIRP_render, AI_render, SU_render, LTA_render], ["TKO.PA", "STM.PA", "THEP.PA", "BIM.PA", "ERF.PA", "TRI.PA","VIRP.PA", "AI.PA", "SU.PA", "LTA.PA"])
euronext = Exchange('euronext', service=execute_order)(price_streams[0], price_streams[1], price_streams[2], price_streams[3], price_streams[4], price_streams[5], price_streams[6], price_streams[7], price_streams[8], price_streams[9])

all_scaled_features = pd.concat([TKO_scaled_features, STM_scaled_features, THEP_scaled_features, BIM_scaled_features, ERF_scaled_features, TRI_scaled_features, VIRP_scaled_features, AI_scaled_features, SU_scaled_features, LTA_scaled_features], axis=1)
with NameSpace("euronext"):
    features = [Stream.source(list(all_scaled_features[feature]), dtype="float").rename(feature) for feature in all_scaled_features.columns]
all_features_feed = DataFeed(features)
all_features_feed.compile()

In [64]:
EUR = Instrument('EUR', 4, 'Euro')
TKO = Instrument('TKO', 4, 'Tikehau')
STM = Instrument('STM', 4, 'STMicroelectronics')
THEP = Instrument('THEP', 4, 'Thermador')
BIM = Instrument('BIM', 4, 'Biomerieux')
ERF = Instrument('ERF', 4, 'Eurofins')
TRI = Instrument('TRI', 4, 'Trigano')
VIRP = Instrument('VIRP', 4, 'Virbac')
AI = Instrument('AI', 4, 'Air Liquide')
SU = Instrument('SU', 4, 'Schneider Electric')
LTA = Instrument('LTA', 4, 'Altamir')

In [65]:
cash = Wallet(euronext, 10000 * EUR)
asset_tko = Wallet(euronext, 0 * TKO)
asset_stm = Wallet(euronext, 0 * STM)
asset_thep = Wallet(euronext, 0 * THEP)
asset_bim = Wallet(euronext, 0 * BIM)
asset_erf = Wallet(euronext, 0 * ERF)
asset_tri = Wallet(euronext, 0 * TRI)
asset_virp = Wallet(euronext, 0 * VIRP)
asset_ai = Wallet(euronext, 0 * AI)
asset_su = Wallet(euronext, 0 * SU)
asset_lta = Wallet(euronext, 0 * LTA)

In [66]:
portfolio = Portfolio(EUR, [cash, asset_tko, asset_stm, asset_thep, asset_bim, asset_erf, asset_tri, asset_virp, asset_ai, asset_su, asset_lta])

reward_scheme = RiskAdjustedReturns(return_algorithm='sortino', window_size=75)
action_scheme = ManagedRiskOrders()

In [67]:
chart_renderer = PlotlyTradingChart(
    display=True,  # show the chart on screen (default)
    height=800,  # affects both displayed and saved file height. None for 100% height.
    save_format="html",  # save the chart to an HTML file
    auto_open_html=True,  # open the saved HTML chart in a new browser tab
)
    
renderer_feed = DataFeed([
    Stream.source(list(LTA_render["LTA.PA:date"])).rename("date"),
    Stream.source(list(LTA_render["LTA.PA:Open"]), dtype="float").rename("open"),
    Stream.source(list(LTA_render["LTA.PA:High"]), dtype="float").rename("high"),
    Stream.source(list(LTA_render["LTA.PA:Low"]), dtype="float").rename("low"),
    Stream.source(list(LTA_render["LTA.PA:Close"]), dtype="float").rename("close"), 
    Stream.source(list(LTA_render["LTA.PA:Volume"]), dtype="float").rename("volume"),
])

In [68]:
env = default.create(
    portfolio=portfolio,
    action_scheme=action_scheme,
    reward_scheme=reward_scheme,
    feed=all_features_feed,
    renderer_feed=renderer_feed,
    renderer=chart_renderer,
    window_size=75,
    max_allowed_loss=0.1
)

In [52]:
def create_env(config):
    TKO_data = pd.read_pickle(config['TKO_filename'])
    STM_data = pd.read_pickle(config['STM_filename'])
    THEP_data = pd.read_pickle(config['THEP_filename'])
    BIM_data = pd.read_pickle(config['BIM_filename'])
    ERF_data = pd.read_pickle(config['ERF_filename'])
    TRI_data = pd.read_pickle(config['TRI_filename'])
    VIRP_data = pd.read_pickle(config['VIRP_filename'])
    AI_data = pd.read_pickle(config['AI_filename'])
    SU_data = pd.read_pickle(config['SU_filename'])
    LTA_data = pd.read_pickle(config['LTA_filename'])
    
    TKO_render, TKO_scaled_features = separate_render_features(TKO_data, 'TKO.PA')
    STM_render, STM_scaled_features = separate_render_features(STM_data, 'STM.PA')
    THEP_render, THEP_scaled_features = separate_render_features(THEP_data, 'THEP.PA')
    BIM_render, BIM_scaled_features = separate_render_features(BIM_data, 'BIM.PA')   
    ERF_render, ERF_scaled_features = separate_render_features(ERF_data, 'ERF.PA')
    TRI_render, TRI_scaled_features = separate_render_features(TRI_data, 'TRI.PA')
    VIRP_render, VIRP_scaled_features = separate_render_features(VIRP_data, 'VIRP.PA')
    AI_render, AI_scaled_features = separate_render_features(AI_data, 'AI.PA')       
    SU_render, SU_scaled_features = separate_render_features(SU_data, 'SU.PA')
    LTA_render, LTA_scaled_features = separate_render_features(LTA_data, 'LTA.PA')     
    
    price_streams = get_price_stream([TKO_render, STM_render, THEP_render, BIM_render, ERF_render, TRI_render, VIRP_render, AI_render, SU_render, LTA_render], ["TKO.PA", "STM.PA", "THEP.PA", "BIM.PA", "ERF.PA", "TRI.PA","VIRP.PA", "AI.PA", "SU.PA", "LTA.PA"])
    euronext = Exchange('euronext', service=execute_order)(price_streams[0], price_streams[1], price_streams[2], price_streams[3], price_streams[4], price_streams[5], price_streams[6], price_streams[7], price_streams[8], price_streams[9])

    all_scaled_features = pd.concat([TKO_scaled_features, STM_scaled_features, THEP_scaled_features, BIM_scaled_features, ERF_scaled_features, TRI_scaled_features, VIRP_scaled_features, AI_scaled_features, SU_scaled_features, LTA_scaled_features], axis=1)
    with NameSpace("euronext"):
        features = [Stream.source(list(all_scaled_features[feature]), dtype="float").rename(feature) for feature in all_scaled_features.columns]
    all_features_feed = DataFeed(features)
    all_features_feed.compile()

    EUR = Instrument('EUR', 4, 'Euro')
    TKO = Instrument('TKO', 4, 'Tikehau')
    STM = Instrument('STM', 4, 'STMicroelectronics')
    THEP = Instrument('THEP', 4, 'Thermador')
    BIM = Instrument('BIM', 4, 'Biomerieux')
    ERF = Instrument('ERF', 4, 'Eurofins')
    TRI = Instrument('TRI', 4, 'Trigano')
    VIRP = Instrument('VIRP', 4, 'Virbac')
    AI = Instrument('AI', 4, 'Air Liquide')
    SU = Instrument('SU', 4, 'Schneider Electric')
    LTA = Instrument('LTA', 4, 'Altamir')
    
    cash = Wallet(euronext, 10000 * EUR)
    asset_tko = Wallet(euronext, 0 * TKO)
    asset_stm = Wallet(euronext, 0 * STM)
    asset_thep = Wallet(euronext, 0 * THEP)
    asset_bim = Wallet(euronext, 0 * BIM)
    asset_erf = Wallet(euronext, 0 * ERF)
    asset_tri = Wallet(euronext, 0 * TRI)
    asset_virp = Wallet(euronext, 0 * VIRP)
    asset_ai = Wallet(euronext, 0 * AI)
    asset_su = Wallet(euronext, 0 * SU)
    asset_lta = Wallet(euronext, 0 * LTA)
    
    portfolio = Portfolio(EUR, [cash, asset_tko, asset_stm, asset_thep, asset_bim, asset_erf, asset_tri, asset_virp, asset_ai, asset_su, asset_lta])

    reward_scheme = RiskAdjustedReturns(return_algorithm='sortino', window_size=75)
    action_scheme = ManagedRiskOrders()

    chart_renderer = PlotlyTradingChart(
        display=True,  # show the chart on screen (default)
        height=800,  # affects both displayed and saved file height. None for 100% height.
        save_format="html",  # save the chart to an HTML file
        auto_open_html=True,  # open the saved HTML chart in a new browser tab
    )
    
    renderer_feed = DataFeed([
        Stream.source(list(TKO_render["TKO.PA:date"])).rename("date"),
        Stream.source(list(TKO_render["TKO.PA:Open"]), dtype="float").rename("open"),
        Stream.source(list(TKO_render["TKO.PA:High"]), dtype="float").rename("high"),
        Stream.source(list(TKO_render["TKO.PA:Low"]), dtype="float").rename("low"),
        Stream.source(list(TKO_render["TKO.PA:Close"]), dtype="float").rename("close"), 
        Stream.source(list(TKO_render["TKO.PA:Volume"]), dtype="float").rename("volume"),
    ])

    env = default.create(
        portfolio=portfolio,
        action_scheme=action_scheme,
        reward_scheme=reward_scheme,
        feed=all_features_feed,
        renderer_feed=renderer_feed,
        renderer=chart_renderer,
        window_size=config["window_size"],
        max_allowed_loss=config["max_allowed_loss"]
    )
    
    return env


In [69]:
#75
window_size = 75
#570
n_steps = 570
def get_optimal_batch_size(window_size=window_size, n_steps=n_steps, batch_factor=4, stride=1):
    """
    lookback = 30          # Days of past data (also named window_size).
    batch_factor = 4       # batch_size = (sample_size - lookback - stride) // batch_factor
    stride = 1             # Time series shift into the future.
    """
    lookback = window_size
    sample_size = n_steps
    batch_size = ((sample_size - lookback - stride) // batch_factor)
    return batch_size

batch_size = get_optimal_batch_size(window_size=window_size, n_steps=n_steps, batch_factor=4)
batch_size

123

In [54]:
env1 = create_env({
    "TKO_filename": "TKO_X_train.pickle", 
    "STM_filename": "STM_X_train.pickle",  
    "THEP_filename": "THEP_X_train.pickle", 
    "BIM_filename": "BIM_X_train.pickle",  
    "ERF_filename": "ERF_X_train.pickle", 
    "TRI_filename": "TRI_X_train.pickle",  
    "VIRP_filename": "VIRP_X_train.pickle", 
    "AI_filename": "AI_X_train.pickle",  
    "SU_filename": "SU_X_train.pickle", 
    "LTA_filename": "LTA_X_train.pickle",  
    "max_allowed_loss": 0.10, 
    "window_size": 75 
})

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U12'), dtype('float64')) -> None

In [ ]:
 #(1029 days - 4 trading years)
memory_capacity = n_steps * 10
n_bins = 5             # Number of bins to partition the dataset evenly in order to evaluate class sparsity.
seed = 1337
commission = 0.001
save_path = 'agents/'

agent = DQNAgent(env)

agent.train(batch_size=batch_size, 
            n_steps=n_steps, 
            n_episodes=50, 
            memory_capacity=memory_capacity, 
            save_path=save_path)


FigureWidget({
    'data': [{'close': array([12.14769173, 11.89601326, 11.76178455, ..., 21.23782158, 21.32916…

In [79]:
import quantstats as qs
def print_quantstats_full_report(env, output='dqn_quantstats'):
    performance = pd.DataFrame.from_dict(env.action_scheme.portfolio.performance, orient='index')
    net_worth = performance['net_worth'].iloc[window_size:]
    returns = net_worth.pct_change().iloc[1:]

    qs.reports.full(returns)
    qs.reports.html(returns, output=output + '.html')

print_quantstats_full_report(env1)

ModuleNotFoundError: No module named 'quantstats'

In [29]:
from keras.models import load_model
model = load_model('agents/policy_network__48f884a__20221213_175108.hdf5')
test = DQNAgent(env1, policy_network = model)

In [33]:
loaded_ag = DQNAgent(env1).restore('agents/policy_network__48f884a__20221213_175108.hdf5')

In [36]:
test.__dict__.keys()

dict_keys(['env', 'n_actions', 'observation_shape', 'policy_network', 'target_network', '_id'])

In [44]:
test.test()

AttributeError: 'DQNAgent' object has no attribute 'test'

In [ ]:
#https://www.tensortrade.org/en/latest/examples/setup_environment_tutorial.html
#https://github.com/tensortrade-org/tensortrade/blob/master/examples/train_and_evaluate.ipynb
https://levelup.gitconnected.com/portfolio-allocation-with-tensortrade-part-2-2-9ac30a6bcbfe
https://www.tensortrade.org/en/latest/agents/overview.html#stable-baselines
https://levelup.gitconnected.com/portfolio-allocation-with-tensortrade-part-2-2-9ac30a6bcbfe
    https://github.com/Tomas0413/tensortrade-experiments/blob/main/TensorTrade%20-%20Sinewave%20with%20SimpleProfit%20and%20ManagedRiskOrders.ipynb
    https://www.google.com/search?q=feature_engine&sourceid=chrome&ie=UTF-8
        https://github.com/tensortrade-org/tensortrade/blob/master/examples/train_and_evaluate.ipynb

In [61]:
pd.DataFrame(env1.action_scheme.portfolio.performance)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
euronext:/EUR-TKO,20.176781,20.271067,20.082495,19.799644,19.42251,19.23394,18.95109,18.762522,18.385386,18.2911,18.385386,18.621096,18.52681,18.2911,17.913965,17.961105,18.149673,18.573954,18.856806,18.573954,18.52681,17.725395,17.913965,18.47967,17.913965,17.913965,18.149673,17.913965,18.008249,18.008249,18.573954,18.196815,18.196815,18.243959,18.856806,18.95109,18.856806,19.045374,19.045374,19.328226,...,19.705362,19.988214,19.705362,19.516792,19.611076,19.799644,19.89393,19.988214,19.611076,19.045374,18.668236,18.95109,19.42251,19.611076,20.365349,20.648203,20.836771,21.025335,21.119621,21.308189,21.213907,21.213907,21.308189,21.308189,21.213907,21.213907,21.308189,21.308189,21.119621,21.213907,21.213907,20.553917,20.742485,20.931055,21.213907,21.119621,21.025335,20.836771,20.836771,20.742485
euronext:/EUR-STM,11.522116,11.757457,11.752653,11.747849,11.675453,11.689929,11.612704,11.274847,11.612704,11.438951,11.458257,11.810597,12.051923,11.723718,10.362627,10.52673,10.956294,10.9756,11.593398,11.747849,11.858862,11.733371,11.882994,12.114669,11.892646,12.413914,12.471834,12.047096,12.066402,13.287524,13.871538,13.581944,13.355095,13.403359,13.384053,13.422666,13.466105,13.345442,13.987374,13.364748,...,13.659166,13.596421,13.485413,13.441972,14.112866,13.812386,13.957776,13.788154,14.39396,14.083786,13.540984,13.696071,12.789783,12.339064,12.760704,13.308352,13.67184,14.447271,14.670209,15.082153,15.164546,15.028843,15.120929,15.217856,15.561954,15.377788,15.707347,15.97875,16.095062,15.528027,16.322845,16.051447,15.717039,15.658883,15.872127,15.891515,15.920587,15.135467,14.864064,15.082153
euronext:/EUR-THEP,39.101494,39.634697,39.723564,39.456963,39.368095,39.456963,39.901299,38.923759,38.657162,38.746025,39.101494,39.723564,39.545834,38.923759,38.212826,38.834896,39.279228,41.589771,42.38958,43.278248,41.767506,41.234306,40.523369,41.589771,41.056572,44.433517,43.633717,44.433517,45.677662,44.433517,43.278248,40.78997,43.189384,43.544849,44.078049,43.367115,44.078049,43.367115,43.100513,41.056572,...,43.722588,44.433517,45.322193,45.322193,45.322193,44.255787,43.01165,42.38958,43.189384,43.455986,43.900318,42.922783,44.255787,45.144459,44.433517,44.966721,45.855392,45.499924,46.210861,48.34367,47.988205,47.277264,47.099529,47.099529,47.632732,47.632732,47.988205,47.988205,47.277264,48.885334,47.966434,47.047543,47.231319,47.415096,46.679981,47.598877,47.231319,47.782661,48.333996,48.333996
euronext:/EUR-BIM,59.576847,60.55191,60.84444,58.504269,57.919224,57.139164,56.359112,54.408966,54.116447,53.531406,55.091515,54.408966,56.066589,54.408966,51.776276,54.798996,56.261604,56.066589,57.139164,57.529202,56.554127,56.554127,57.041668,58.504269,58.406761,59.86937,60.161888,59.674351,59.86937,60.356903,61.722,59.47934,60.746929,59.674351,60.161888,59.47934,60.064381,63.379627,62.014526,62.014526,...,70.790161,72.057747,70.887672,70.887672,70.400131,71.277702,72.252777,71.960258,71.472717,70.985184,70.107605,72.252777,71.08268,71.08268,71.86274,73.471611,74.202911,75.568016,74.885468,75.81179,75.470505,75.031715,75.080475,75.031715,75.177979,75.470505,73.812881,70.985184,70.595154,72.204025,67.279915,67.767433,68.108719,68.645012,68.98629,68.98629,69.961349,68.98629,69.132538,69.57132
euronext:/EUR-ERF,32.583515,33.731232,33.692326,32.097198,31.513609,31.280174,30.657684,30.521513,30.774401,30.677137,30.443705,30.968931,31.708139,31.844311,30.638231,31.747046,32.252815,34.159199,34.412083,34.47044,33.264366,32.641876,32.11665,32.427891,32.194462,34.99567,34.95676,35.365273,34.80114,34.976215,36.882595,35.015121,34.295364,34.042484,34.198105,33.556156,33.809044,33.458897,32.11665,32.038837,...,35.073483,35.579258,35.987766,36.104481,35.481991,35.890503,36.629707,35.268005,35.773781,35.015121,3

In [63]:
pd.DataFrame(env1.reward_scheme.)

ValueError: DataFrame constructor not properly called!

In [72]:
env1.reward_scheme

AttributeError: 'RiskAdjustedReturns' object has no attribute '_Clock'

In [55]:
test.predict()

AttributeError: 'DQNAgent' object has no attribute 'predict'

In [ ]:
def print_quantstats_full_report(env, data, output='dqn_quantstats'):
    performance = pd.DataFrame.from_dict(env.action_scheme.portfolio.performance, orient='index')
    net_worth = performance['net_worth'].iloc[window_size:]
    returns = net_worth.pct_change().iloc[1:]

    # WARNING! The dates are fake and default parameters are used!
    returns.index = pd.date_range(start=data['date'].iloc[0], freq='1d', periods=returns.size)

    qs.reports.full(returns)
    qs.reports.html(returns, output=output + '.html')

print_quantstats_full_report(env, data)